# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - należy zaimplementować w tym celu prostą wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # <<< HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [107]:
from typing import Tuple

from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove
from copy import copy
import numpy as np
from math import inf



Wielkość planszy

In [108]:
ROW_COUNT = 6
COLUMN_COUNT = 7

In [109]:
class Max(Player):
    def __init__(self, char: str) -> None:
        super().__init__(char)

class Min(Player):
    def __init__(self, char: str) -> None:
        super().__init__(char)

In [110]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour) -> None:
        """
        Constructor requires that a game consists of one Max player and one Min player,
        otherwise Solver could not calculate optimal moves;
        attributes column_count and row_count are stored for code readability
        """
        if type(game.first_player) is not Max and type(game.first_player) is not Min:
            raise ValueError("The first player is not Max nor Min")
        if type(game.second_player) is not Max and type(game.second_player) is not Min:
            raise ValueError("The second player is not Max nor Min")
        if type(game.first_player) is type(game.second_player):
            raise ValueError("Both players are the same type; One should be Max and one should be Min")
        self.game = game
        self.column_count = len(self.game.state.fields)
        self.row_count = len(self.game.state.fields[0])
        

    def evaluate_position(self, player: Player) -> float:
        """
        check if game is finished
        if so, check is there is a winner and return propoer value
        if not, evaluate the position from the perspective of the player
        who made the last move 
        (since in the that's how the algorithm determines the best move for a given player)
        """
        if self.game.is_finished():
            winner = self.game.get_winner()
            if type(winner) is Max:
                return inf
            elif type(winner) is Min:
                return -inf
            else:
                return 0.0
        else:
            evaluation = self._evaluate_score(player)
            return evaluation

    def _evaluate_score(self, player: Player) -> float:
        """
        Heuristic function that checks in how many ways 
        a player and his opponent can still win.
        I.E. how many lines of four are not blocked by his opponent.
        The function returns the difference of victory possibilities between the player and
        his opponent
        """
        opponent = self.game.get_players()[0]
        if opponent is player:
            opponent = self.game.get_players()[1]
        score = 0.0
        score += self._check_columns(player, opponent)
        score += self._check_rows(player, opponent)
        score += self._check_diagonals(player, opponent)

        #If player is of type Min, return the -score (as the Min player wants to minimize the score)
        if type(player) is Min:
            score = -score
        return score
    
    """
    _check_columns(),  _check_rows() and check_diagonals() calculate the difference
    of victory possibilites between players in colums, rows and diagonals. 
    """
    def _check_columns(self, player: Player, opponent: Player) -> float:
        score = 0.0
        for column in range(self.column_count):
            for starting_row in range(self.row_count - 3):
                four_fields = self.game.state.fields[column][starting_row:starting_row+4]
                score += self._score_four_fields(player, four_fields)
                score -= self._score_four_fields(opponent, four_fields)
        return score
    
    def _check_rows(self, player: Player, opponent: Player) -> float:
        score = 0.0
        for row in range(self.row_count):
            for starting_column in range(self.column_count - 3):
                four_fields = [self.game.state.fields[starting_column + i][row] for i in range(4)]
                score += self._score_four_fields(player, four_fields)
                score -= self._score_four_fields(opponent, four_fields)
        return score
    
    def _check_diagonals(self, player: Player, opponent: Player) -> float:
        score = 0.0
        for starting_column in range(self.column_count - 3):
            for starting_row in range(self.row_count - 3):
                four_fields = [self.game.state.fields[starting_column + i][starting_row + i] for i in range(4)]
                score += self._score_four_fields(player, four_fields)
                score -= self._score_four_fields(opponent, four_fields)
                four_fields = [self.game.state.fields[starting_column + i][self.row_count - 1 - starting_row - i] for i in range(4)]
                score += self._score_four_fields(player, four_fields)
                score -= self._score_four_fields(opponent, four_fields)
        return score
    
    def _score_four_fields(self, player: Player, four_fields: list) -> float:
        """
        a method that defines if a player can use defined four fields to win,
        i.e. the opponent has not taken any field in the given fields.
        """
        if four_fields.count(player) + four_fields.count(None) == 4:
            return 1
        return 0

    def is_valid_move(self, col_index:int)->bool:
        return self.game.state.fields[col_index][ROW_COUNT - 1] is None
    
    def get_valid_moves(self) -> list:
        """Retruns the list of valid moves in curreent position"""
        valid_moves = []
        for column in range(COLUMN_COUNT):
            if self.is_valid_move(column):
                valid_moves.append(column)
        return valid_moves

    def minimax(self, depth, alpha:float, beta:float, is_maximizing_player:bool)-> Tuple[int, float]:
        """
        MinMax algorith with alpha-beta pruning; 
        Checks if the game is finished or the given depth is 0.
        If so, calls evaluate_position
        and if not, calls _evaluate_next_state() for every valid move.
        _evaluate_next_state() method calls minimax() with depth argument
        decremented by one, and is_maximizing_player reversed.
        The algorithm then evaluates leads to the best possible outcome.
        It also uses alpha beta pruning to reduce unnecessary calculations.
        
        Returns column index and score
        """
        if self.game.is_finished() or depth == 0:
            player = self.game.get_players()[1] #player that makes the last move in the evaluated postion
            return None, self.evaluate_position(player)
        else:
            valid_moves = self.get_valid_moves()
            best_move = np.random.choice(valid_moves, 1)[0]
            if is_maximizing_player:
                evaluation = -inf
                for move in valid_moves:
                    new_evaluation = self._evaluate_next_state(move, depth, alpha, beta, is_maximizing_player)
                    if new_evaluation > evaluation:
                        evaluation = new_evaluation
                        best_move = move
                    alpha = max(alpha, evaluation)
                    if alpha >= beta:
                        break
                return best_move, alpha
            else:
                evaluation = inf
                for move in valid_moves:
                    new_evaluation = self._evaluate_next_state(move, depth, alpha, beta, is_maximizing_player)
                    if new_evaluation < evaluation:
                        evaluation = new_evaluation
                        best_move = move
                    beta = min(beta, evaluation)
                    if alpha >= beta:
                        break
                return best_move, beta
    def _evaluate_next_state(self, move: int, depth: int, alpha:float, beta: float, is_maximizing_player: bool) -> float:
        evaluated_game = copy(self.game)
        evaluated_game.make_move(ConnectFourMove(move))
        new_solver = MinMaxSolver(evaluated_game)
        return new_solver.minimax(depth-1, alpha, beta, not is_maximizing_player)[1]    
            


Rozgrywka

In [129]:

player_one = Max("x")
player_two = Min("o")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=player_one, second_player=player_two)
game.make_move(ConnectFourMove(6))
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(6))
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(6))
game.make_move(ConnectFourMove(2))
print(game.state)
s = MinMaxSolver(game)
depth_one = s.minimax(1, -inf, inf, True)
depth_two = s.minimax(2, -inf, inf, True)
print(f"Best move with depth 1: {depth_one[0]}, evaluation: {depth_one[1]}")
print(f"Best move with depth 2: {depth_two[0]}, evaluation: {depth_two[1]}")
print(s.minimax(2, -inf, inf, True))
game.make_move(ConnectFourMove(5))
print("Player one made move: 5")
print(game.state)
depth_one = s.minimax(1, -inf, inf, False)
depth_two = s.minimax(2, -inf, inf, False)
print(f"Best move with depth 1: {depth_one[0]}, evaluation: {depth_one[1]}")
print(f"Best move with depth 2: {depth_two[0]}, evaluation: {depth_two[1]}")

Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][x]
[ ][ ][ ][o][ ][ ][x]
[ ][ ][o][o][ ][ ][x]
Best move with depth 1: 6, evaluation: inf
Best move with depth 2: 6, evaluation: inf
(6, inf)
Player one made move: 5
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][x]
[ ][ ][ ][o][ ][ ][x]
[ ][ ][o][o][ ][x][x]
Best move with depth 1: 3, evaluation: -16.0
Best move with depth 2: 6, evaluation: 2.0


In [134]:

player_one = Max("x")
player_two = Min("o")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=player_one, second_player=player_two)
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(0))
game.make_move(ConnectFourMove(4))
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(5))
game.make_move(ConnectFourMove(5))

print(game.state)
s = MinMaxSolver(game)
depth_two = s.minimax(2, -inf, inf, True)
depth_three = s.minimax(3, -inf, inf, True)
print(f"Best move with depth 2: {depth_two[0]}, evaluation: {depth_two[1]}")
print(f"Best move with depth 3: {depth_three[0]}, evaluation: {depth_three[1]}")
game.make_move(ConnectFourMove(5))
print("Player one made move: 5")
print(game.state)

depth_one = s.minimax(1, -inf, inf, False)
depth_two_first = s.minimax(2, -inf, inf, False)
depth_two_second = s.minimax(2, -inf, inf, False)
print(f"Best move with depth 1: {depth_one[0]}, evaluation: {depth_one[1]}")
print(f"Best move with depth 2 (first try): {depth_two_first[0]}, evaluation: {depth_two_first[1]}")
print(f"Best move with depth 2 (second try): {depth_two_second[0]}, evaluation: {depth_two_second[1]}")


Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][o][ ][o][ ]
[o][ ][ ][x][x][x][ ]
Best move with depth 2: 2, evaluation: inf
Best move with depth 3: 0, evaluation: inf
Player one made move: 5
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][x][ ]
[ ][ ][ ][o][ ][o][ ]
[o][ ][ ][x][x][x][ ]
Best move with depth 1: 3, evaluation: -11.0
Best move with depth 2 (first try): 2, evaluation: inf
Best move with depth 2 (second try): 4, evaluation: inf


In [113]:
def play_game(depth_one: int, depth_two: int):
    """
    Play a game between two algorithms with given depths
    """
    player_one = Max("a")
    player_two = Min("b")
    game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=player_one, second_player=player_two)
    solver = MinMaxSolver(game)
    while not game.is_finished():
        move, evaluation = solver.minimax(depth_one, -inf, inf, type(solver.game.get_current_player()) is Max)
        solver.game.make_move(ConnectFourMove(move))
        print(f"Player one move: {move}, evaluation {evaluation}")
        print(game.state)
        if game.get_winner():
            break
        move, evaluation = solver.minimax(depth_two, -inf, inf, type(solver.game.get_current_player()) is Max)
        solver.game.make_move(ConnectFourMove(move))
        print(f"Player two move: {move}, evaluation: {evaluation}")
        print(game.state)
    


In [114]:
play_game(6, 4)

Player one move: 3, evaluation -0.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: 6.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 2, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][a][a][ ][ ][ ]
Player two move: 1, evaluation: 5.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][b][a][a][ ][ ][ ]
Player one move: 3, evaluation -3.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][b][a][a][ ][ ][ ]
Player two move: 3, evaluation: 4.0
Current player: a
[ ][ ][ ][ ][ ][

In [115]:
play_game(7, 5)

Player one move: 3, evaluation 5.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -0.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 2, evaluation 3.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][a][a][ ][ ][ ]
Player two move: 1, evaluation: -2.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][b][a][a][ ][ ][ ]
Player one move: 3, evaluation 3.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][b][a][a][ ][ ][ ]
Player two move: 3, evaluation: -3.0
Current player: a
[ ][ ][ ][ ][ ][

In [116]:
play_game(8, 5)

Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -0.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 1, evaluation -3.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][a][ ][a][ ][ ][ ]
Player two move: 2, evaluation: -3.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][a][b][a][ ][ ][ ]
Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][a][b][a][ ][ ][ ]
Player two move: 3, evaluation: -3.0
Current player: a
[ ][ ][ ][ ][

In [117]:
play_game(4, 4)

Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: 6.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -0.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: 2.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: 3.0
Current player: a
[ ][ ][ ][b][ ][

In [118]:
play_game(5,5)

Player one move: 1, evaluation 6.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][a][ ][ ][ ][ ][ ]
Player two move: 1, evaluation: -4.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][b][ ][ ][ ][ ][ ]
[ ][a][ ][ ][ ][ ][ ]
Player one move: 1, evaluation 6.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][a][ ][ ][ ][ ][ ]
[ ][b][ ][ ][ ][ ][ ]
[ ][a][ ][ ][ ][ ][ ]
Player two move: 1, evaluation: -3.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][b][ ][ ][ ][ ][ ]
[ ][a][ ][ ][ ][ ][ ]
[ ][b][ ][ ][ ][ ][ ]
[ ][a][ ][ ][ ][ ][ ]
Player one move: 3, evaluation 5.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][b][ ][ ][ ][ ][ ]
[ ][a][ ][ ][ ][ ][ ]
[ ][b][ ][ ][ ][ ][ ]
[ ][a][ ][a][ ][ ][ ]
Player two move: 4, evaluation: -1.0
Current player: a
[ ][ ][ ][ ][ ][

In [119]:
play_game(4, 5)

Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -0.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -0.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -2.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -3.0
Current player: a
[ ][ ][ ][b][

In [120]:
play_game(4, 6)

Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: 5.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -0.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: 3.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: 2.0
Current player: a
[ ][ ][ ][b][ ][

In [121]:
play_game(4, 7)

Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -2.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -0.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -3.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -2.0
Current player: a
[ ][ ][ ][b][

In [122]:
play_game(1, 1)

Player one move: 3, evaluation 7.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -3.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation 9.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -2.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation 6.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -0.0
Current player: a
[ ][ ][ ][b][ ][

In [123]:
play_game(4, 1)

Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -3.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -0.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -2.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -0.0
Current player: a
[ ][ ][ ][b][

In [124]:
play_game(6, 6)

Player one move: 3, evaluation -0.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: 5.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 2, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][a][a][ ][ ][ ]
Player two move: 4, evaluation: 3.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][a][a][b][ ][ ]
Player one move: 3, evaluation -2.0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][a][a][b][ ][ ]
Player two move: 3, evaluation: 2.0
Current player: a
[ ][ ][ ][ ][ ][

In [125]:
def play_with_computer(depth: int):
    """play a game agains an algorith with given depth"""
    player_one = Max("a")
    player_two = Min("b")
    game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=player_one, second_player=player_two)
    solver = MinMaxSolver(game)
    while not game.is_finished():
        print(game.state)
        move = int(input("Enter your move: "))
        if solver.is_valid_move(move):
            solver.game.make_move(ConnectFourMove(move))
        else:
            continue
        print(f"Player one move: {move}")
        if game.get_winner():
            break
        print(game.state)
        move, evaluation = solver.minimax(depth, -inf, inf, type(solver.game.get_current_player()) is Max)
        solver.game.make_move(ConnectFourMove(move))
        print(f"Player two move: {move}, evaluation: {evaluation}")
    print(game.state)
        

In [127]:
play_with_computer(3)

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Player one move: 3
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -2.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player two move: 3, evaluation: -0.0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Player one move: 3
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][

# Wnioski

Siła algorytmu jest w dużej mierze uzależniona od wybranej heurystycznej metody oceny pozycji. Dlatego też solver z większą głębokośćią przeszukiwań nie zawsze pokona (a czasami nawet może przegrać) z solverem o mniejszej głębokości (wykorzystywana heurystyka nie zawsze prowadzi do pozycji faktycznie korzystniejszej). Jednak większa głębokość obliczeń zazwyczaj daje przewagę nad przeciwnikiem, gdyż we wcześniejszych fazach może zdefiniować które ruchy prowadzą do nieuniknionego zwycięstwa lub nieuniknionej porażki.

Stworzony algorytm radzi sobie świetnie, jeśli jest w stanie obliczyć, że istnieje ścieżka, która prowadzi go do pewngo zwycięstwa, jednak ze względu na implementację, nie zawsze wybierze ścieżkę najkrótszą. Analogicznie, w sytuacji nieuniknionej przegranej; algorytm zacznie wykonywać losowe legalne ruchy, co może nie zmusić przeciwnika do wykonania najdłuższej serii ruchów prowadzących do zwycięstwa. Dzieje się tak, gdyż algorytm zakłada, że jego przeciwnik jest równie dobry, tzn. oblicza tyle samo ruchów do przodu, co on sam, przez to algorytm teoretycznie lepszy, może przegrać z algorytmem słabszym (algorytm lepszy zacznie wykonywać losowe ruchy, zamiast przeanalizować pozycje z punktu widzenia przeciwnika i pozwolić mu na popełnienie błędu)
